In [ ]:
#for dev purpose
import os
USERNAME = os.getenv('USERNAME').lower()
new_pull = False

In [ ]:
# import python packages
import sys
if sys.version_info[1]==12:
    os.add_dll_directory(r'C:/Program Files/Python/312/Lib/site-packages/clidriver/bin')
else:
    pass
import pandas as pd
import datetime as dt
import time
import warnings
from dateutil.relativedelta import relativedelta
from snp_query_box import DsnpHelperFunction, hotMacQueries
from snp_query_box.dsnp_transform import dsnpTransform, callLogTransform

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
print(f"User: {USERNAME} Started creating hot_mac_call_df in the shared storage")

In [ ]:
# set the starting time
start_time = time.time()

In [ ]:
# calculate the dates
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
last_date_of_last_month = DsnpHelperFunction.last_date_of_last_month(today_str)
first_date_of_month = dt.datetime.today().replace(day=1)
first_date_of_last_month = first_date_of_month - relativedelta(months=1)

call_start_date = pd.to_datetime(today - relativedelta(months = 3)).strftime('%Y-%m-%d')
print(call_start_date)

reporting_end_date = last_date_of_last_month_str
print(reporting_end_date)

In [ ]:
call_start_date

In [ ]:
# set the storage and output path
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
output_path = f'{storage_path}/{reporting_end_date}'

isExist = os.path.exists(output_path)
print(output_path)
if not isExist:
    # Create a new directory because it does not exist
     os.makedirs(output_path)
     print("The new directory is created!")
else:
      print("The folder already exist")
print("Output files will sit in the folder above.")

In [ ]:
#locate data dump folder
if new_pull:
    pull_hot_mac_call = hotMacQueries.pull_hot_mac_call(min_call_date=call_start_date, max_call_date=today_str)
    pull_hot_mac_call.to_parquet(f"{output_path}/pull_hot_mac_call_3month.parquet")
else:
    try:
        pull_hot_mac_call = pd.read_parquet(f"{output_path}/pull_hot_mac_call_3month.parquet")

    except:
        pull_hot_mac_call = hotMacQueries.pull_hot_mac_call(min_call_date=call_start_date, max_call_date=today_str)

        pull_hot_mac_call.to_parquet(f"{output_path}/pull_hot_mac_call_3month.parquet")

In [ ]:
hot_mac_call_df = callLogTransform.clean_hot_mac_call_log(pull_hot_mac_call)

In [ ]:
print(hot_mac_call_df.CALL_TIMESTAMP.min())
print(hot_mac_call_df.CALL_TIMESTAMP.max())

In [ ]:
print(hot_mac_call_df.shape)
print(hot_mac_call_df.head())

In [ ]:
hot_mac_call_df.to_parquet(f"{output_path}/hot_mac_call_df_3month.parquet")

In [ ]:
# output the process time
print("Process time -- %s seconds" % (time.time() - start_time))